In [0]:
%md

# For QA Test
## Note book for QA Test


In [0]:


    
from applications.db_check_v1.common.constants import query, citus_settings

import unittest
from pyspark.sql.functions import count
import datetime

def datetime_to_string(date_time, convert_format="%Y-%m-%d"):
    return datetime.datetime.strftime(date_time, convert_format)

def check_date_completeness():
    numdays=329
    start_date='2019-01-01'
    end_date='2019-11-26'
    base = datetime.date.today()- datetime.timedelta(days=2)
    expected_date_list = [str(base - datetime.timedelta(days=x)) for x in range(numdays)]
    # print expected_date_list
    t = unittest.TestCase('run')

    date_fail = dict()
    actual_date_list = []
    country_code_list = ['US', 'GB', 'CA', 'AU', 'IN', 'JP', 'KR']
    language_list=["en","ja","ko"]
    device_code='ios-all'
    for country_code in country_code_list:
        print country_code
        for language in language_list:
            date_fail[device_code] = list()
            sql = "SELECT distinct date FROM  advancedreview.{} where country_code='{}' and language='{}' and device_code='ios-all' and date between '{}' and '{}'".format("advancedreview_term_fact_v3", country_code, language, start_date, end_date)
            date_list = query(citus_settings("aa"), sql)
            for item in date_list:
                actual_date_list.append(datetime_to_string(item[0]))
            for date in expected_date_list:
                if str(date) not in actual_date_list:
                    date_fail[device_code].append(date)
            if not date_fail[device_code]:
                date_fail.pop(device_code)
            t.assertFalse(date_fail, msg='{}'.format(date_fail))
    print 'pass'

check_date_completeness()


In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=advancedreview;
--EXPLAIN ANALYZE
select date, country_code, count(1) from advancedreview_term_fact_v3 where date between '2020-12-04' and '2020-12-05' group by date, country_code order by date, country_code desc
--COMMIT;
EOF 



In [0]:
%%sh

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=advancedreview;
select date,country_code,count(*) from advancedreview_term_fact_v3 where date between '2019-12-24' and '2019-12-26' group by date,country_code order by date;
EOF 


In [0]:
%%sh
# PGPASSWORD='@7EFiOthPf96' psql -h 10.2.10.132 -p 5432 -Ucitus1_datapipeline_advance -d aa_citus_db << EOF
# SET search_path=advancedreview;
# BEGIN;
# SELECT  * , array_length(term_ids,1)  FROM   advancedreview_term_fact_v1 where product_id='431946152' ;
# COMMIT;
# EOF 


# #485624 444924

# SELECT *  FROM   advancedreview_topic_fact_v1 where date='2019-07-08' and country_code='AU' limit 4;

# PGPASSWORD='@7EFiOthPf96' psql -h 10.2.10.132 -p 5432 -Ucitus1_datapipeline_advance -d aa_citus_db << EOF
# SET search_path=advancedreview;
# BEGIN;
# SELECT * FROM  advancedreview_topic_fact_v1 where language='ko' order by date desc limit 3; 
# COMMIT;
# EOF

# PGPASSWORD='@7EFiOthPf96' psql -h 10.2.10.132 -p 5432 -Ucitus1_datapipeline_advance -d aa_citus_db << EOF
# SET search_path=advancedreview;
# BEGIN;
# SELECT count(*)  FROM   advancedreview_topic_fact_v1 where date='2019-07-02' and country_code='US' and  language='en';
# COMMIT;
# EOF
#  and 0.0 > ANY(scores) limit 4;
# SELECT review_id FROM advancedreview_term_fact_v1_p_201812 LIMIT 10;
# SELECT DISTINCT product_id FROM advancedreview_term_fact_v1_p_201812 WHERE review_id='3596923019' LIMIT 10;


PGPASSWORD='@7EFiOthPf96' psql -h 10.2.10.132 -p 5432 -Ucitus1_datapipeline_advance -d aa_citus_db << EOF
SET search_path=advancedreview;
\timing
\d advancedreview_term_fact_v1_p_201812;


BEGIN;
EXPLAIN ANALYZE

select max(term) as term,
ROUND((pow(sum(edf), 2) * (AVG(rating) - 4.5) * AVG(LN(score)) / count(*))::numeric, 1) as fact_review_phrase_impact_score__aggr,
count(*) OVER() AS total
from (
    select advancedreview_term_fact_v1.review_id as review_id,
    unnest(advancedreview_term_fact_v1.terms) as term,
    unnest(advancedreview_term_fact_v1.term_ids) as term_id,
    unnest(advancedreview_term_fact_v1.scores) as score,
    unnest(advancedreview_term_fact_v1.is_extracted) as edf,
    advancedreview_term_fact_v1.rating as rating
    from advancedreview_term_fact_v1
    INNER JOIN advancedreview_topic_fact_v1
    ON advancedreview_topic_fact_v1.review_id = advancedreview_term_fact_v1.review_id
    and advancedreview_topic_fact_v1.product_id = advancedreview_term_fact_v1.product_id
    where
        advancedreview_term_fact_v1.product_id = 284882215
    AND advancedreview_term_fact_v1.country_code = 'CA'
    AND advancedreview_topic_fact_v1.product_id = 284882215
    AND advancedreview_topic_fact_v1.country_code = 'CA'
    and advancedreview_topic_fact_v1.topic_ids @> ARRAY[10]::smallint[]) as sub_table
GROUP BY term_id;


COMMIT;
EOF 




In [0]:

from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg

# result = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/date=2019-09-11/")
# result.printSchema()
# result.show()
# row = result.collect()
# array_result= row[0].asDict()["terms"]
# print len(array_result)

# result=result.withColumn("record_date", result['review_datetime'].cast(DateType()))
# result=result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
# result.groupBy("record_date").agg(count("count_review_id").alias("total_review")).show()


result = spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/")
# result.filter("country=AU and language=en'" ).select("review_id").count().show()
# ,result.process_date.between('2019-07-01','2019-07-15')
x = result.filter("country='AU' and language='en' and ( process_date >= '2019-07-01' and  process_date <= '2019-07-15' ) ").withColumn("record_date", result['review_datetime'].cast(DateType())).select("record_date").distinct()
# result.filter("country='AU' and language='en' and ( process_date >= '2019-07-01' and  process_date <= '2019-07-15' )").groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
# print x # (658702)
x.show()

print '~~~'
# result = spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/process_date=2019-07-14/country=AU/language=en/")

# result.filter("review_id='4458376928'").show()

# result1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/")
# y = result1.filter("country_code='AU' and language='en' and ( process_date >= '2019-07-01' and  process_date <= '2019-07-15' ) ").select("review_id").distinct().count()
# print y
# result1.filter("process_date='2019-07-02' and country_code='JP' and language='en' and product_version is null ").show(30,False)
# and (process_date >= '2019-07-01' and  process_date <= '2019-07-15')
# review_id='4401037413'
# result=result.withColumn("record_date", result['review_datetime'].cast(DateType()))
# result=result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
# result.groupBy("record_date").agg(count("count_review_id").alias("total_review")).show()



In [0]:
%%sh
# SELECT *  FROM   advancedreview_topic_fact_v1 where date='2019-07-08' and country_code='AU' limit 4;

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=advancedreview;
BEGIN;
SELECT count(*) FROM  advancedreview_term_fact_v3 where _identifier='120200213104705259';
COMMIT;
EOF

In [0]:

df1=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-02-12").show()
print df1
# 54392
# df1=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/process_date=2020-02-03").filter("review_id=5458214386")

# df2=spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2020-02-03").filter("review_id='5458214386'").show(20,False)

#  5197363743 | 1331794412 | 1.9.2           | US           | en       |      1 | 2019-11-26 | {3985275,330954,16940738,452239,1794980} | {"12 year","age rating 4","cheating boyfriends","sex relationships","sexual ads"} | {4,9,4,4,3} | {1,1,1,1,1}  | ios-all     | 220191128034836095 |                    0


In [0]:

spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2019-12-03/country=US/language=en/").show()


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/process_date=2020-02-03/
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/process_date=2020-02-04/

aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-02-02/
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-02-03/
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-02-04/


 

In [0]:


from pyspark.sql import Row
from pyspark.sql.types import DateType

# term_df =  spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/process_date=2019-10-17/country=US/language=en")
# term_df.createOrReplaceTempView("term_table")
# results = spark.sql("select * from term_table where review_id='4956824744' ")

# print type(results)

# result_data=results.rdd.collect()
# for x in result_data:
#     print x
    


# term_df =  spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/process_date=2019-10-17/country=US/language=en/")
# term_df.createOrReplaceTempView("term_table")
# # results = spark.sql("select term_table.term,term_table.value,term_table.is_extracted from term_table where review_id='4956824744' ")
# results = spark.sql("select count(is_extracted) from term_table where review_id='4956824744' ")
# results.show()
# print results.collect()[0][0]
# print results["is_extracted"].values[0]
# print type(results)

# result1=results.rdd.collect()
# for x in result_data:
#     print x
#     print x["is_extracted"]


df=spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/").filter("review_id='4997107774'").select("term","value","is_extracted")
# df=df.withColumn("record_date", df['review_datetime'].cast(DateType()))
# # result1.show()
# df.createOrReplaceTempView("term_table")
# result1 = spark.sql("select term from term_table where review_id='4993373879' and record_date='2019-10-21' ")
# result1.printSchema()
# result1.show()
result1 = df.collect()
# array_result= row[0].asDict()["terms"]
# print len(array_result)

#  4993373879 |  870161082 | 6.4             | US           | en       |      1 | 2019-10-21 | {158,6759038}       | {"latest version","open check-ins"}           | {3.25,3.5} | {1,1}        | ios-all     |                    0


# result1=result1.collect()
# print result1[1][0]
# spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/process_date=2019-10-17/country=US/language=en/").filter("term is null").show()

result2=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/").filter("review_id='4997107774'").select("terms","scores","is_extracted")
# df2.createOrReplaceTempView("term_table_unified")
# result2 = spark.sql("select terms from term_table_unified where review_id='4993373879' and date='2019-10-21'")
# result2.printSchema()
# result2.show()

result2 = result2.collect()
# array_result= row[0].asDict()["terms"]

# print len(array_result)


# print result2[0][0]
# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/date=2019-10-17/").filter("product_id='407690035'").show(20,False)


# spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/process_date=2019-06-30/country=US/language=en/").filter("review_id='4389282703'").show()

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/date=2019-07-08/part-20190708.c000.gz.parquet").filter("country_code='US' and review_id='4427349889'").show()


def map_list_sequences(row_rdd1_original, row_rdd2_to_list):
    # print row_rdd1_original
    # print row_rdd2_to_list
    # print 
    original_line_length=len(row_rdd1_original)
    
    print original_line_length
    if 1:
        # print 'current to_list_index is ' , to_list_index
        range_length=len(row_rdd2_to_list[0])
        for line in range(0,original_line_length):
            target_value = row_rdd1_original[line][0]
            # print 'range_length: ',range_length

            for x in range(0,range_length):
                to_list_index=row_rdd2_to_list[0][0].index(target_value)
                print 'to list index: ', to_list_index

                print row_rdd1_original[line][x]
                print row_rdd2_to_list[0][x][to_list_index]

                # print row_rdd1_original[line][x]
                # print row_rdd2_to_list[0][x][to_list_index]
 
                # print row_rdd1_original[line][x]
                # print row_rdd2_to_list[0][x][to_list_index]


map_list_sequences(result1,result2)

In [0]:
_
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta

def get_date_list(start_date, end_date):
    r = []
    start = string_to_datetime(start_date)
    delta = get_total_days(start_date, end_date)
    for i in range(delta):
        r.append(datetime_to_string(start + timedelta(days=i)))
    return r



def get_date_list_from_granularity(start_date, end_date, granularity):
    date_list = get_date_list(start_date, end_date)
    if granularity == 'weekly':
        _date_list = []
        for date in date_list:
            if string_to_datetime(date).isoweekday() % 7 == 6:
                _date_list.append(date)
        date_list = _date_list
    elif granularity == 'monthly':
        _date_list = set()
        for date in date_list:
            _, last_day = get_month_start_end_date(date)
            _date_list.add(last_day)
        date_list = list(_date_list)
    return date_list


def get_date_list_test():
    start_date='2019-01-01'
    end_date='2019-10-21'
    date_list = get_date_list_from_granularity(
    start_date, end_date, 'daily')
    print date_list

def string_to_datetime(date_str, convert_format=None):
    return datetime.strptime(date_str, convert_format if convert_format else "%Y-%m-%d")


def datetime_to_string(date_time, convert_format="%Y-%m-%d"):
    return datetime.strftime(date_time, convert_format)


def get_total_days(start_date, end_date):
    start = string_to_datetime(start_date)
    end = string_to_datetime(end_date)
    delta = end - start
    return delta.days + 1

date_time_list=get_date_list_test()


In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-raw-advancedreview/raw/app-int.advancedreview.term.v1/insert/_default_partition=2019-10/ | sort -n
# aws s3 ls s3://b2c-prod-data-pipeline-raw-advancedreview/raw/app-int.advancedreview.topic.v1/


# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/date=2019-10-23/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/


# python  ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py readavro -k raw/app-int.advancedreview.topic.v1/insert/_default_partition=2019-10/aidp6ca09cc6d5e34736fc96af1e4da34db1.avro

# aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2019-12-03/country=US/
# aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2019-12-25/country=KR/language=en/
aws s3 ls --recursive s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2019-12-26/| grep -v "_SUCCESS"
# aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2019-12-26/country=US/language=en/ 

 

In [0]:
%%sh
export PYTHONPATH=$SPARK_HOME/python/:$SPARK_HOME/python/lib/py4j-0.10.7-src.zip:$PYTHONPATH

# python  ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py readavro -k raw/app-int.advancedreview.topic.v1/insert/_default_partition=2019-10/aidp6ca09cc6d5e34736fc96af1e4da34db1.avro
# python  ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py readavro -k raw/app-int.advancedreview.topic.v1/insert/_default_partition=2019-10/aidp2ba2f19739e3616c96bdfd846f7a0aa3.avro
# python ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py listdynamoitems -t B2C-PROD-DATA-PIPELINE_key_id_mapping  -g app-int.advancedreview -id "auto-ingest-app-int.advancedreview.term.vv1_term_travel constantly"

var=0
while read line; do
echo $var 
if [ $var -gt 289 ] 
then
python  ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py readavro -k raw/app-int.advancedreview.term.v1/insert/_default_partition=2019-10/$line   >> /tmp/notification_date_3
fi
var=$((var + 1))

done < '/tmp/avro_file'



In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-raw-advancedreview/raw/app-int.advancedreview.term.v1/insert/_default_partition=2019-12/| sort


 

In [0]:
%%sh
export PYTHONPATH=$SPARK_HOME/python/:$SPARK_HOME/python/lib/py4j-0.10.7-src.zip:$PYTHONPATH
python  ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py readavro -k raw/app-int.advancedreview.term.v1/insert/_default_partition=2019-12/aidpb3a57c8c6fe625ec1ec6a51cb27dfb9a.avro




In [0]:


# aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/
# spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/").show()

# df = spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/").printSchema()

columns = ['version','platform','process_date','country','language','review_id', 'app_id', 'app_version', 'rating', 'review_datetime' ,'term','value','is_extracted' ]
vals = [("1.0.0","2","2019-10-23","AU","en","4996479491","542090992" ,"3.1.3" ,"1", "2019-10-23T00:00:00.000000Z" ,"abc def","8.333333333333334","1")]

(
    spark.createDataFrame(vals, columns)
    .write.mode("append")
    .parquet(
        's3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS',
        partitionBy=['version','platform','process_date','country','language'],
        compression="gzip"
    )
)
    
path = "s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2/process_date=2019-10-23/country=AU/language=en"    
# appended = df.union(df_new)
# appended.show()


# newRow = spark.createDataFrame([(4,5,7)], columns)
# appended = df.union(newRow)

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/").filter("country_code='GB' and language='en' and product_id='1441365703' and review_id='4874637878'").show()


In [0]:
%%sh
export PYTHONPATH=$SPARK_HOME/python/:$SPARK_HOME/python/lib/py4j-0.10.7-src.zip:$PYTHONPATH
python  ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py readavro -k raw/app-tech.aso.keyword-trafficshare.v1/insert/_default_partition=2019-11/aidp6be578525c2e54dd11f1bc7d2dafc9aa.avro

# 2019-11-15 14:58:58       2490 aidp84d3d61c90d502ce9989d0ad9d66c822.avro
# 2019-11-17 10:10:20       2491 aidpdd88a27a39fdf65e28bd0d00267b2dcc.avro
# 2019-11-18 01:51:57       2491 aidp6be578525c2e54dd11f1bc7d2dafc9aa.avro


In [0]:

from pyspark.sql.functions import array_contains
df=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/")
# df.printSchema()
# df=df.withColumn("contain_stop_word_data",df.select(array_contains(df.terms,"quality")))
df.groupBy("review_id").agg(count("review_id")).alias("count_review_id").sort("count_review_id.count(review_id)",ascending=False).show()
# df.filter("array_contains(terms, quality)==true").show()
# df.filter("review_id='3730118274'").show()


In [0]:

import unittest
raw_process_date_path = "s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.0/platform=2" \
                        "/process_date=2019-07-13/"
unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview" \
                            ".term.v1/"
                            

def check_data_in_process_date_group_by():
    t = unittest.TestCase('run')
    raw_result = spark.read.parquet(raw_process_date_path).filter("country='AU' and language='en'")
    raw_result = raw_result.withColumn("record_date", raw_result['review_datetime'].cast(DateType()))
    raw_result = raw_result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
    raw_data_row = sorted(raw_result.groupBy("record_date").agg(count("count_review_id").alias("total_review")).collect())

    unified_result = spark.read.parquet(unified_process_date_path)
    unified_result = unified_result.withColumn("record_date", unified_result['date'].cast(DateType()))
    unified_data_row = sorted(unified_result.filter(
    "process_date='2019-07-13' and country_code='AU' and language='en'").groupBy(
    "date").count().collect())

    range_len = len(raw_data_row)
    print range_len
    for x in range(0, range_len):
        print raw_data_row[x][0]
        print 
        t.assertEqual(raw_data_row[x][0], unified_data_row[x][0],
                     'date is not equals {} ~ {}, debug info {} \n {} '.format(raw_data_row[x][0],
                                                                               unified_data_row[x][0],
                                                                               raw_process_date_path,
                                                                               unified_process_date_path))
        t.assertEqual(raw_data_row[x][1], unified_data_row[x][1],
                     'number is not equals {} ~ {}, debug info {} \n {} '.format(raw_data_row[x][1],
                                                                               unified_data_row[x][1],
                                                                               raw_process_date_path,
                                                                               unified_process_date_path))
    print 'no error'
                                                                                   
                                                                                   
check_data_in_process_date_group_by()

In [0]:

import unittest
from pyspark.sql.functions import count
from pyspark.sql.functions import col

def count_review_id_test_term():
    t = unittest.TestCase('run')
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]

    result = spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/")
    for language in language_list:

        for country in country_list:
            print language, country
            raw_count = result.filter("country='{}' and language='{}' and ( process_date >= '2019-01-01' and  process_date <= '2019-11-25' )".format(country,language)).withColumnRenamed("review_id", "raw_review_id").withColumnRenamed("process_date","raw_process_date").select("raw_review_id").distinct().count()
            result1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/")
            unified_count = result1.filter("country_code='{}' and language='{}' and ( process_date >= '2019-01-01' and  process_date <= '2019-11-25' ) ".format(country,language)).select("review_id").distinct().count()
            # df=unified_count.join(raw_count, [raw_count.raw_review_id==unified_count.review_id], "outer")
            # df.filter(col("raw_review_id").isNull()).show() #groupBy("process_date").agg({"review_id":"count"}).show()
            print 'raw count is : ', raw_count
            print 'unified_count is : ', unified_count
            t.assertEqual(raw_count,unified_count, "raw data is not equals with unifed data {} - {} ".format(raw_count,unified_count))
    
count_review_id_test_term()

In [0]:
%%sh
PGPASSWORD='@7EFiOthPf96' psql -h 10.2.10.132 -p 5432 -Ucitus1_datapipeline_advance -d aa_citus_db << EOF
SET search_path=advancedreview;
select * from advancedreview_term_fact_v3 where review_id='5189725805';
COMMIT;
EOF 

# %pyspark
# df=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/").filter("country_code='US'").where(" review_id='5189725805' ").orderBy("review_id")
# df.show()


SET
 review_id  | product_id | product_version | country_code | language | rating |    date    | term_ids  |      terms      | scores | is_extracted | device_code |    _identifier     | _disable_idx_4_query 
------------+------------+-----------------+--------------+----------+--------+------------+-----------+-----------------+--------+--------------+-------------+--------------------+----------------------
 5189725805 | 1016814119 |                 | US           | en       |      5 | 2019-11-24 | {4274061} | {"cute mascot"} | {4}    | {1}          | ios-all     | 220191128034836095 |                    0
(1 row)


+------------------+------------+----------+-----------+--------------------+------------+--------+----------+---------------+------+----------+------+---------+-------------+------------+
|       _identifier|country_code|      date|device_code|              doc_id|is_extracted|language|product_id|product_version|rating| review_id|scores| term_ids|        terms|process_date|
+------------------+------------+----------+-----------+--------------------+------------+--------+----------+---------------+------+----------+------+---------+-------------+------------+
|220191128034836095|          US|2019-11-24|    ios-all|2019-11-25-518972...|         [1]|      en|1016814119|           null|     5|5189725805| [4.0]|[4274061]|[cute mascot]|  2019-11-25|
+------------------+------------+----------+-----------+--------------------+------------+--------+----------+---------------+------+----------+------+---------+-------------+------------+


# %pyspark
# from pyspark.sql.functions import array_contains
# df=spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/").filter("review_id='5189725805'").show()

+------------------+------------+----------+-----------+--------------------+------------+--------+----------+---------------+------+----------+------+---------+-------------+------------+
|       _identifier|country_code|      date|device_code|              doc_id|is_extracted|language|product_id|product_version|rating| review_id|scores| term_ids|        terms|process_date|
+------------------+------------+----------+-----------+--------------------+------------+--------+----------+---------------+------+----------+------+---------+-------------+------------+
|220191128034836095|          US|2019-11-24|    ios-all|2019-11-25-518972...|         [1]|      en|1016814119|           null|     5|5189725805| [4.0]|[4274061]|[cute mascot]|  2019-11-25|
+------------------+------------+----------+-----------+--------------------+------------+--------+----------+---------------+------+----------+------+---------+-------------+------------+



In [0]:
%%sh
export PYTHONPATH=$SPARK_HOME/python/:$SPARK_HOME/python/lib/py4j-0.10.7-src.zip:$PYTHONPATH
python ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py listdynamoitems -t B2C-PROD-DATA-PIPELINE_key_id_mapping  -g app-int.advancedreview -id "auto-ingest-app-int.advancedreview.term.vv1_term_cute mascot"


In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg
from datetime import datetime

test_date = ['2019-08-0'+ str(a) for a in xrange(1,10)] + ['2019-08-1'+str(a) for a in xrange(0,10)] +  ['2019-08-2'+str(a) for a in xrange(0,10)]

                            
def check_data_in_process_date_group_by(pd):
    print pd
    raw_process_date_path = "s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2" \
                        "/process_date={}/".format(pd)
    unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview" \
                            ".term.v1/fact/process_date={}/".format(pd)

    t = unittest.TestCase('run')
    raw_result = spark.read.parquet(raw_process_date_path).filter("country='US' and language='en'")
    raw_result = raw_result.withColumn("record_date", raw_result['review_datetime'].cast(DateType()))
    raw_result = raw_result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
    raw_data_row = raw_result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id")).collect()

    # raw_data_row = raw_result.groupBy("record_date").agg(count("count_review_id").alias("total_review")).collect()

    unified_result = spark.read.parquet(unified_process_date_path)
    unified_result = unified_result.withColumn("record_date", unified_result['date'].cast(DateType()))
    unified_data_row = unified_result.filter(
    "country_code='US' and language='en'".format(pd)).collect()

    range_len = len(raw_data_row)
    range_len_unified=len(unified_data_row)
    review_id_list = []
    for x in range(0,range_len):
        # print raw_data_row[x][1]
        review_id_list.append(raw_data_row[x][1])
    print len(review_id_list)

    print '~~~~~~~~~~~'
    review_id_list_unified = []
    print range_len_unified
    for x in range(0,range_len_unified):
        review_id_list_unified.append(unified_data_row[x][10])
    print len(review_id_list_unified)
    l= [x for x  in review_id_list if x not in review_id_list_unified]
    print '!!!', len(l) , "different is : " , l 
    for x in l:
        row=unified_result.filter("review_id={}".format(x)).select("process_date").collect()[0]
        _date = datetime.strptime(pd, "%Y-%m-%d").date()
        if(row[0]<_date):
            print 'compare current process date' , pd , ' ~~~ unified process date',  row
         
    print 'no error'
                                                                                   
for x in test_date:
    check_data_in_process_date_group_by(x)

In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg

unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v2/fact/"
def check_rating_value():
    t = unittest.TestCase('run')
    unified_result = spark.read.parquet(unified_process_date_path)
    unified_result = unified_result.withColumn("record_date", unified_result['date'].cast(DateType()))
    unified_data_row = unified_result.filter(
        "process_date='2020-02-03' and country_code='US' and language='en'").collect()

    range_len = len(unified_data_row)
    for x in range(0, range_len):
        array_result= unified_data_row[x].asDict()["rating"]
        t.assertTrue(1<=array_result<=5, "rating is not correct")
    print 'pass'
        
check_rating_value()


In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg

unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v2/fact/"
def check_score_is_larger_than_zero():
    unittest.TextTestRunner(verbosity=2)
    t = unittest.TestCase('run')
    unified_result = spark.read.parquet(unified_process_date_path)
    unified_result = unified_result.withColumn("record_date", unified_result['date'].cast(DateType()))
    unified_data_row = unified_result.filter(
        "process_date='2020-02-03' and country_code='US' and language='en'").collect()

    range_len = len(unified_data_row)
    for x in range(0, range_len):
        array_result= unified_data_row[x].asDict()["scores"]
        for i in array_result:
            t.assertTrue(i>0, "score is not correct")
    print 'pass'
check_score_is_larger_than_zero()


In [0]:
%%sh
aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/

In [0]:

# from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
#     get_aso_citus_connection)
# from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
# from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
#     get_citus_connection)
import unittest
from pyspark.sql.functions import count


def test_count():
    t = unittest.TestCase('run')

    
    sql="SELECT distinct count(*) FROM  advancedreview.advancedreview_term_fact_v3  where device_code='ios-all' and country_code='{}' and language='{}' ;"

    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]

    result = spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/")
    for language in language_list:

        for country in country_list:
            print language, country
            raw_count = result.filter(" process_date ='2020-02-03' ").filter("country='{}' and language='{}' ".format(country,language)).select("review_id").distinct().count()
            # db_result = get_citus_connection(urn, sql.format(country,language))
            unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/"
            unified_data_count=spark.read.parquet(unified_process_date_path).filter("process_date='2020-02-03'").filter("country_code='{}' and language='{}' ".format(country,language)).select("review_id").distinct().count()
            print 'raw count is : ', raw_count
            # print 'db is : ', db_result[0][0]
            print 'unified data' ,unified_data_count
            t.assertEqual(raw_count,unified_data_count, "raw data is not equals with unifed data {} - {} ".format(raw_count,unified_data_count))
            # t.assertEqual(raw_count,db_result[0][0], "raw data is not equals with DB data {} - {} ".format(raw_count,db_result[0][0]))



test_count()

In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg

unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/"
def check_is_extracted_is_zero_or_one():
    unittest.TextTestRunner(verbosity=2)
    t = unittest.TestCase('run')
    unified_result = spark.read.parquet(unified_process_date_path)
    unified_result = unified_result.withColumn("record_date", unified_result['date'].cast(DateType()))
    unified_data_row = unified_result.filter(
        "process_date='2020-02-03' and country_code='US' and language='en'").collect()

    range_len = len(unified_data_row)
    for x in range(0, range_len):
        array_result= unified_data_row[x].asDict()["is_extracted"]
        for i in array_result:
            t.assertTrue(i in [0 ,1], "is_extracted value is not correct ")
    print 'pass'
check_is_extracted_is_zero_or_one()


In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_citus_connection)
import unittest
from pyspark.sql.functions import count

def test_count_for_term():
    t = unittest.TestCase('run')
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]
    sql="SELECT count(distinct review_id) FROM  advancedreview.advancedreview_term_fact_v3  where country_code='{}' and language='{}' and device_code='ios-all';"
    raw_path="s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/"
    for language in language_list:
        for country in country_list:
            result = spark.read.parquet(raw_path).filter("country_code='{}' and language='{}'".format(country,language)).select("review_id").distinct().count()
            print language, country
            db_result = get_citus_connection(urn, sql.format(country,language))
            print 'unified count is : ', result
            print 'db is : ', db_result[0][0]
            t.assertEqual(result,db_result[0][0], "unified data is not equals with unifed data {} - {} ".format(result,db_result[0][0]))


test_count_for_term()

In [0]:

# from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
#     get_aso_citus_connection)
# from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
# from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
#     get_citus_connection)
    
from applications.db_check_v1.common.constants import query, citus_settings

import unittest
from pyspark.sql.functions import count
import datetime

def datetime_to_string(date_time, convert_format="%Y-%m-%d"):
    return datetime.datetime.strftime(date_time, convert_format)

def check_date_completeness():
    numdays=329
    start_date='2019-01-01'
    end_date='2019-11-26'
    base = datetime.date.today()- datetime.timedelta(days=2)
    expected_date_list = [str(base - datetime.timedelta(days=x)) for x in range(numdays)]
    # print expected_date_list
    t = unittest.TestCase('run')

    date_fail = dict()
    actual_date_list = []
    country_code_list = ['US', 'GB', 'CA', 'AU', 'IN', 'JP', 'KR']
    language_list=["en","ja","ko"]
    device_code='ios-all'
    for country_code in country_code_list:
        print country_code
        for language in language_list:
            date_fail[device_code] = list()
            sql = "SELECT distinct date FROM  advancedreview.{} where country_code='{}' and language='{}' and device_code='ios-all' and date between '{}' and '{}'".format("advancedreview_term_fact_v3", country_code, language, start_date, end_date)
            date_list = query(citus_settings("aa"), sql)
            for item in date_list:
                actual_date_list.append(datetime_to_string(item[0]))
            for date in expected_date_list:
                if str(date) not in actual_date_list:
                    date_fail[device_code].append(date)
            if not date_fail[device_code]:
                date_fail.pop(device_code)
            t.assertFalse(date_fail, msg='{}'.format(date_fail))
    print 'pass'

check_date_completeness()
